# Solving the input type problem for CaImAn
## UPDATE: this is the wrong way, maybe impossible to work with a numpy array. A shame. A new attempt is now to export the file as multipage tiff or hdf5 before working with caiman
It doesn't support opening nd2 directly. It does support tif (multipage recommended for large files (>2GB)) and memmapped files, but nd2 arrays can be also used as input (see https://caiman.readthedocs.io/en/master/core_functions.html?highlight=load#movie-handling)
The latter is being experimented with here

In [ ]:
def printMethods(data_object) -> None:
    """
    Prints out methods that can be used with the parameter object.
    Useful for:
        ND2_READER objects (returned from pims_nd2.ND2_READER)
        ABF objects (returned from abf.ABF)
    """
    for method in dir(data_object):
        if(method.startswith("_")):
            continue #do not print "magic" attributes (__xy__) and those intended for internal use (_xy)
        else:
            print(method)

In [2]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [1]:
import caiman
import numpy as np
import pims_nd2  # pip install pims_nd2
import nd2_to_caiman

ModuleNotFoundError: No module named 'caiman'

In [4]:
nd2_fpath = 'D:/PhD/Data/T386_MatlabTest/T386_20211202_green.nd2'
nd2_large_fpath = 'D:/PhD/Data/T301/d1/anfall/T301_tmev_d1.270820.1151.nd2'

In [5]:
# now nd2_to_caiman.py 
def np_arr_from_nd2(nd2_fpath: str):
    # set iter_axes to "t"
    # then: create nd array with sizes matching frame size,
    with pims_nd2.ND2_Reader(nd2_fpath) as nikon_file:  # todo: get metadata too?
        
        sizes_dict = nikon_file.sizes
        sizes = (sizes_dict['t'], sizes_dict['x'], sizes_dict['y'])
        
        # dtype would be float32 by default...
        frames_arr = np.zeros(sizes, dtype=nikon_file.pixel_type)
        
        # TODO: probably it is not even necessary to export an np.array, as nikon_file is an iterable of
        #  subclasses of np array... not sure what caiman needs
        for i_frame, frame in enumerate(nikon_file):
            frames_arr[i_frame] = np.array(nikon_file[0], dtype=nikon_file.pixel_type)  # not sure if dtype needed here
        return frames_arr

In [10]:
n = np_arr_from_nd2(nd2_large_fpath)  # 

In [7]:
n.shape  # dimensions match

(16485, 512, 512)

In [8]:
n[0].shape  # first frame

(512, 512)

In [ ]:
# TODO: check if the image is rotated! Not sure what rotation caiman expects (it should not matter, but result should)
#  be same orientation as input file... maybe frames_arr[i] = np.array(...) creates transpose when accessing indices